<a href="https://colab.research.google.com/github/yvishyst/DS-Unit-1-Sprint-3-Statistical-Tests-and-Experiments/blob/master/module3-introduction-to-bayesian-inference/LS_DS_133_Introduction_to_Bayesian_Inference_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lambda School Data Science Module 133

## Introduction to Bayesian Inference




## Assignment - Code it up!

Most of the above was pure math - now write Python code to reproduce the results! This is purposefully open ended - you'll have to think about how you should represent probabilities and events. You can and should look things up, and as a stretch goal - refactor your code into helpful reusable functions!

Specific goals/targets:

1. Write a function `def prob_drunk_given_positive(prob_drunk_prior, prob_positive, prob_positive_drunk)` that reproduces the example from lecture, and use it to calculate and visualize a range of situations
2. Explore `scipy.stats.bayes_mvs` - read its documentation, and experiment with it on data you've tested in other ways earlier this week
3. Create a visualization comparing the results of a Bayesian approach to a traditional/frequentist approach
4. In your own words, summarize the difference between Bayesian and Frequentist statistics

If you're unsure where to start, check out [this blog post of Bayes theorem with Python](https://dataconomy.com/2015/02/introduction-to-bayes-theorem-with-python/) - you could and should create something similar!

Stretch goals:

- Apply a Bayesian technique to a problem you previously worked (in an assignment or project work) on from a frequentist (standard) perspective
- Check out [PyMC3](https://docs.pymc.io/) (note this goes beyond hypothesis tests into modeling) - read the guides and work through some examples
- Take PyMC3 further - see if you can build something with it!

In [0]:
def prob_drunk_given_positive(prob_drunk_prior,prob_positive,prob_positive_drunk):
  #Applying bayes therom
  return (prob_positive_drunk*prob_positive/prob_drunk_prior)


def repeat_bayes_thereom(prob_positive_prior, prob_false_positive, prob_false_negative, num_iterations):
  curr_positive_prior = prob_positive_prior
  for i in range(num_iterations):
    prob_positive_prior = (1-prob_false_positive)*curr_positive_prior/((1-prob_false_positive)*(curr_positive_prior)+(prob_false_negative)*(1-curr_positive_prior))
    curr_positive_prior = prob_positive_prior
    print("pp:",curr_positive_prior,"\t cp:",prob_positive_prior)
  return curr_positive_prior

In [12]:
repeat_bayes_thereom(0.005,0.01,0.01,9)

pp: 0.33221476510067116 	 cp: 0.33221476510067116
pp: 0.9801000000000001 	 cp: 0.9801000000000001
pp: 0.9997949506335922 	 cp: 0.9997949506335922
pp: 0.9999979283737839 	 cp: 0.9999979283737839
pp: 0.9999999790744397 	 cp: 0.9999999790744397
pp: 0.9999999997886307 	 cp: 0.9999999997886307
pp: 0.999999999997865 	 cp: 0.999999999997865
pp: 0.9999999999999785 	 cp: 0.9999999999999785
pp: 0.9999999999999998 	 cp: 0.9999999999999998


0.9999999999999998

Object `sc.stats.bayes_mvs` not found.


## Resources

- [Worked example of Bayes rule calculation](https://en.wikipedia.org/wiki/Bayes'_theorem#Examples) (helpful as it fully breaks out the denominator)
- [Source code for mvsdist in scipy](https://github.com/scipy/scipy/blob/90534919e139d2a81c24bf08341734ff41a3db12/scipy/stats/morestats.py#L139)